<a href="https://colab.research.google.com/github/faisalaldwaish/Sentiment-Analysis-of-Restaurant-Reviews/blob/main/full_review_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install transformers gradio


In [11]:
pip install fuzzywuzzy[speedup]


In [29]:
import pandas as pd
from transformers import pipeline
import gradio as gr
import seaborn as sns
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz

# Load the restaurant reviews dataset
reviews_df = pd.read_csv('Restaurant_reviews.csv')

# Load the Hugging Face sentiment analysis model
sentiment_model = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Function to classify a user's review based on dataset matching and sentiment analysis
def classify_review(user_review):
    try:
        if not user_review.strip():
            return "Please enter a valid review."

        best_match = None
        best_score = 0

        for _, row in reviews_df.iterrows():
            if pd.isna(row['Review']):
                continue

            # Use fuzzy matching to find the most similar review in the dataset
            score = fuzz.token_sort_ratio(user_review.lower(), str(row['Review']).lower())
            if score > best_score:
                best_score = score
                best_match = row

        if best_score > 80:  # If the fuzzy match score is greater than 80
            rating = best_match['Rating']
            rating_based_classification = f"Positive review based on rating: {rating}" if int(rating) >= 4 else f"Negative review based on rating: {rating}"

            sentiment_result = sentiment_model(user_review)[0]
            sentiment = sentiment_result['label']
            confidence = sentiment_result['score']
            sentiment_based_classification = f"Model prediction: {sentiment} with confidence: {confidence:.2f}"

            return f"{rating_based_classification}\n{sentiment_based_classification}\nMatching Score: {best_score}%"
        else:
            return "Review not found in the dataset."

    except Exception as e:
        return f"An error occurred: {str(e)}"

# Function to plot the distribution of ratings from the dataset
def plot_rating_distribution():
    plt.figure(figsize=(8, 6))
    sns.countplot(x='Rating', data=reviews_df, order=[1, 2, 3, 4, 5])
    plt.title('Distribution of Ratings')
    plt.xlabel('Rating')
    plt.ylabel('Count')
    plt.tight_layout()
    return plt.gcf()

# Function to allow users to preview the dataset (first 10 rows)
def preview_dataset():
    return reviews_df.head(15)  # Return the first 10 rows of the dataset

# Create the Gradio interface for classifying reviews
review_interface = gr.Interface(
    fn=classify_review,
    inputs=gr.Textbox(lines=2, placeholder="Enter your review here", label="Reviews"),
    outputs="text",
    title="Review Classifier Based on Rating and Hugging Face Model",
    description="Enter a restaurant review. The system will classify it based on the dataset rating and use a sentiment analysis model."
)

# Create the Gradio interface for plotting the rating distribution
plot_interface = gr.Interface(
    fn=plot_rating_distribution,
    inputs=[],
    outputs="plot",
    title="Rating Distribution",
    description="Shows the distribution of ratings in the dataset."
)

# Create the Gradio interface for previewing the dataset
preview_interface = gr.Interface(
    fn=preview_dataset,
    inputs=[],
    outputs="dataframe",
    title="Preview Restaurant Reviews Dataset",
    description="Displays the first 10 rows of the dataset for preview."
)



# Combine all interfaces (Review Classifier, Rating Distribution, Dataset Preview, Dataset Download) into tabs
tabbed_interface = gr.TabbedInterface(
    [review_interface, plot_interface, preview_interface],
    ["Review Classifier", "Rating Distribution", "Dataset Preview", ""]
)

# Launch the Gradio interface
tabbed_interface.launch()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://cad10aea2a73f11c89.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
